## Goals

* Ingest streaming data from one of two sources: either Twitter or Wikipedia edits
* Perform pre-processing and save data to persistent storage

For Twitter data, you'll need to visit [the Twitter App management page](https://dev.twitter.com), create an application if necessary, and fill in your credentials in `twitter4j.properties`.

In [ ]:
%dependency org.apache.spark %% spark-streaming % 1.3.1

In [ ]:
%dependency org.apache.spark %% spark-streaming-twitter % 1.3.1

In [ ]:
%dependency com.google.code.gson % gson % 2.4

In [ ]:
import org.apache.spark.streaming._
import org.apache.spark.streaming.twitter._

import com.google.gson.Gson

In [ ]:
val WikiServerHost  = "ec2-54-213-33-240.us-west-2.compute.amazonaws.com"
val WikiServerPort  = 9002

val BatchInterval   = Seconds(10)

In [ ]:
val ssc = new StreamingContext(spark, BatchInterval)

### Choose a data source

In [ ]:
// Twitter
val dStream = TwitterUtils.createStream(ssc, None)
dStream.foreachRDD { (rdd, time) =>
    if(! rdd.isEmpty) {
        rdd.map{x =>
            var gson = new Gson()
            gson.toJson(x)
        }
           .saveAsTextFile("data/" + time.milliseconds.toString)
    }
}

// Task serializability is why we put the gson declaration inside the map.

In [ ]:
// Wikipedia
val dStream = ssc.socketTextStream(WikiServerHost, WikiServerPort)
dStream.foreachRDD { (rdd, time) =>
    if(! rdd.isEmpty) {
        rdd.saveAsTextFile("data/" + time.milliseconds.toString)
    }
}

Check out [the programming guide](https://spark.apache.org/docs/1.2.0/streaming-programming-guide.html) for useful diagrams of DStreams.

*Question:* How do we decide what logic to put here and what to save till later?

In [ ]:
ssc.start()

In [ ]:
ssc.stop(stopSparkContext = false, stopGracefully = true)
// We'll still have to create a new StreamingContext to try again, though

**Remember to stop the Streaming Context!**